In [1]:
import pandas as pd
import numpy as np

In [13]:
#create empty dataframe with datetime index in range of sample data
feature_df = pd.DataFrame(index=pd.date_range(start='1/1/2015', end='7/1/2020'))

In [14]:
#add special day 'Kieler Woche'
kw_19 = [date for date in pd.date_range(start='22/6/2019', end='30/6/2019',)]
kw_18 = [date for date in pd.date_range(start='16/6/2018', end='24/6/2018',)]
kw_17 = [date for date in pd.date_range(start='17/6/2017', end='25/6/2017',)]
kw_16 = [date for date in pd.date_range(start='18/6/2016', end='26/6/2016',)]
kw_15 = [date for date in pd.date_range(start='20/6/2015', end='28/6/2015',)]

kieler_woche = []
for kw in [kw_15,kw_16,kw_17,kw_18,kw_19]:
    for date in kw:
        kieler_woche.append(date)

In [15]:
#get weather data from csv
wetter = pd.read_csv('feature_data/Wetterdaten_Kiel_gesamt.csv',delimiter=';')

In [16]:
#create datetime index
wetter.index = pd.to_datetime(wetter.pop('Datum'),dayfirst=True)
#drop unknown column
wetter = wetter.loc[wetter.index != pd.to_datetime('1/1/2015')][wetter.columns[:-1]]

In [17]:
#merge weather data with continuous date index
feature_df = pd.merge(left=feature_df,right=wetter,how='left',left_index=True,right_index=True)
#fill missing values with that of next row
feature_df.fillna(method='backfill',inplace=True)

In [20]:
#add school holidays
sommerferien_sh = [[date for date in pd.date_range(start='7/20/2015', end='8/29/2015',)],
                   [date for date in pd.date_range(start='7/25/2016', end='9/3/2016',)],
                   [date for date in pd.date_range(start='7/24/2017', end='9/2/2017',)],
                   [date for date in pd.date_range(start='7/9/2018', end='8/18/2018',)],
                   [date for date in pd.date_range(start='7/1/2019', end='8/10/2019',)],
                   [date for date in pd.date_range(start='6/29/2020', end='8/8/2020',)]
                  ]

herbstferien_sh = [[date for date in pd.date_range(start='10/19/2015', end='10/31/2015',)],
                   [date for date in pd.date_range(start='10/17/2016', end='10/29/2016',)],
                   [date for date in pd.date_range(start='10/16/2017', end='10/27/2017',)],
                   [date for date in pd.date_range(start='10/1/2018', end='10/19/2018',)],
                   [date for date in pd.date_range(start='10/4/2019', end='10/18/2019',)]
                  ]

weihnachtsferien_sh = [[date for date in pd.date_range(start='12/21/2015', end='1/6/2016',)],
                       [date for date in pd.date_range(start='12/23/2016', end='1/6/2017',)],
                       [date for date in pd.date_range(start='12/21/2017', end='1/6/2018',)],
                       [date for date in pd.date_range(start='12/21/2018', end='1/4/2019',)],
                       [date for date in pd.date_range(start='12/23/2019', end='1/6/2020',)]
                      ]

#create lists with datestamps for respective holidays
SoFe_SH = []
HeFe_SH = []
WiFe_SH = []

for holidays in sommerferien_sh:
    for date in holidays:
        SoFe_SH.append(date)

for holidays in herbstferien_sh:
    for date in holidays:
        HeFe_SH.append(date)

for holidays in weihnachtsferien_sh:
    for date in holidays:
        WiFe_SH.append(date)

In [22]:
#create public holidays

karfreitag = [pd.to_datetime(f'4/3/2015'),
              pd.to_datetime(f'3/25/2016'),
              pd.to_datetime(f'4/14/2017'),
              pd.to_datetime(f'3/30/2018'),
              pd.to_datetime(f'4/19/2019'),
              pd.to_datetime(f'4/10/2019')
             ]

ostermontag = [pd.to_datetime(f'4/6/2015'),
               pd.to_datetime(f'3/28/2016'),
               pd.to_datetime(f'4/17/2017'),
               pd.to_datetime(f'4/2/2018'),
               pd.to_datetime(f'4/22/2019'),
               pd.to_datetime(f'4/13/2019')
              ]

himmelfahrt = [pd.to_datetime(f'5/14/2015'),
               pd.to_datetime(f'5/5/2016'),
               pd.to_datetime(f'5/25/2017'),
               pd.to_datetime(f'5/10/2018'),
               pd.to_datetime(f'5/30/2019'),
               pd.to_datetime(f'5/21/2019')
              ]

pfingstmontag = [pd.to_datetime(f'5/25/2015'),
                 pd.to_datetime(f'5/16/2016'),
                 pd.to_datetime(f'6/5/2017'),
                 pd.to_datetime(f'5/21/2018'),
                 pd.to_datetime(f'6/10/2019'),
                 pd.to_datetime(f'6/1/2019')
                ]

tag_der_arbeit = [pd.to_datetime(f'5/1/20{val}') for val in range(15,21)]

einheitstag = [pd.to_datetime(f'10/3/20{val}') for val in range(15,21)]

heilig_abend = [pd.to_datetime(f'12/24/20{val}') for val in range(15,21)]

weihnachten = [pd.to_datetime(f'12/{day}/20{year}') for year in range(15,21) for day in range(25,27)]

silvester = [pd.to_datetime(f'12/31/20{val}') for val in range(15,21)]

neujahr = [pd.to_datetime(f'1/1/20{val}') for val in range(15,21)]



In [23]:
#function to make 1/0 dummy from datestamp lists
def dummy_from_datelist(date_index,datelist):
    return [1 if idx in datelist else 0 for idx in date_index]

In [24]:
feature_df['SoFe_SH'] = dummy_from_datelist(feature_df.index,SoFe_SH)
feature_df['HeFe_SH'] = dummy_from_datelist(feature_df.index,HeFe_SH)
feature_df['WiFe_SH'] = dummy_from_datelist(feature_df.index,WiFe_SH)
feature_df['Kieler_Woche'] = dummy_from_datelist(feature_df.index,kieler_woche)

feature_df['karfreitag'] = dummy_from_datelist(feature_df.index,karfreitag)
feature_df['ostermontag'] = dummy_from_datelist(feature_df.index,ostermontag)
feature_df['himmelfahrt'] = dummy_from_datelist(feature_df.index,himmelfahrt)
feature_df['pfingstmontag'] = dummy_from_datelist(feature_df.index,pfingstmontag)
feature_df['tag_der_arbeit'] = dummy_from_datelist(feature_df.index,tag_der_arbeit)
feature_df['einheitstag'] = dummy_from_datelist(feature_df.index,einheitstag)
feature_df['heilig_abend'] = dummy_from_datelist(feature_df.index,heilig_abend)
feature_df['weihnachten'] = dummy_from_datelist(feature_df.index,weihnachten)
feature_df['silvester'] = dummy_from_datelist(feature_df.index,silvester)
feature_df['neujahr'] = dummy_from_datelist(feature_df.index,neujahr)


In [37]:
#add flag one day before irregular holiday
feature_df['day_before_irregular_holiday'] = [1 if int(feature_df[['karfreitag','ostermontag','himmelfahrt','pfingstmontag']][i-1:i].sum(axis=1)) > 0 else 0 for i in range(1,len(feature_df)+1)]

In [38]:
feature_df.to_csv('feature_data/special_days.csv',index=True,index_label='ds')

In [39]:
#import train, validation and test data
train_df, validation_df, test_df = pd.read_csv('data/train_fil_3.csv',index_col=0),pd.read_csv('data/validation_fil_3.csv',index_col=0),pd.read_csv('data/test_fil_3.csv',index_col=0)
for df in [train_df,validation_df,test_df]:
    df.columns = [int(col) for col in df.columns]
    df.index = pd.to_datetime(df.index)

In [40]:
features = pd.read_csv('feature_data/special_days.csv')
features.index = features.pop('ds')

In [41]:
def merge_events_on_index(df,event_df):
    return pd.merge(left=df.copy(),right=features,how='left',left_index=True,right_index=True)

In [42]:
a = merge_events_on_index(train_df,features)